## Exploring Ebay Car Sales Data

In this guided project, we are going to work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. The aim of this project is to clean the data and analyze the included used car listings. Also, we are going to become familiar with some of the unique benefits jupyter notebook provides for pandas.

The dataset was originally scraped and uploaded to Kaggle and the dateset that gonna be used in this project is based on that with severak modifications. There are in total 50,000 data points sampled from the full dataset to ensure the code running quickly in the hosted environment and the dataset was dirtied a little bit to more closely resemble what people would expect from a scraped dataset since the version uploaded to Kaggle had been alredy cleaned to be easier to work with.

We'll first begin with importing the libraries we need and reading the dataset into pandas.

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding = "Latin-1")
# Change the default encoding as UTF-8 encounters errors

In [2]:
# autos
# Jupyter notebook is able to render the first few and last few values of any pandas object

In [3]:
# Print out the basic information of our imported dataset as well as the first several lines
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


From the basic information, we can observe that, among the 20 columns collected, most of these are string objects and only a few of these are integer object. Meanwhile, some of the columns have null data and it can be up to almost 20%. Last but not least, the column names are camelcase instead of the snakecase, as a result, we cannot replace spaces with underscores.
We will start from converting the format of the column names and rewording some of the names.

In [4]:
# Check all the column names
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [5]:
# Replace some of the 'camelcase' names with 'snakecase' names and reanme some of these
autos.rename({"yearOfRegistration":"registration_year", "monthOfRegistration":"registration_month", 
              "notRepairedDamage":"unrepaired_damage", "dateCreated":"ad_created", "offerType":"offer_type", 
              "vehicleType":"vehicle_type", "fuelType":"fuel_type", "nrOfPictures":"pictures_number", "abtest":"ab_test",
              "postalCode":"postal_code", "lastSeen":"last_seen", "powerPS":"power_in_PS", "dateCrawled":"date_crawled"},
             axis = 1, inplace = True)
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_in_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,pictures_number,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


We converted the format of all column names into snakecase format and rewording some of the names using common abbreviations or to make the names more descriptive. This will help us to deal with the column names more easily using the format Python likes and have these better organized.

In [6]:
# Check the descriptive statistics for different columns
print(autos.describe(include = 'all'), "\n")

print(autos["pictures_number"].describe(), "\n")
      
print(autos["postal_code"].describe())

               date_crawled         name  seller offer_type  price ab_test  \
count                 50000        50000   50000      50000  50000   50000   
unique                48213        38754       2          2   2357       2   
top     2016-03-05 16:57:05  Ford_Fiesta  privat    Angebot     $0    test   
freq                      3           78   49999      49999   1421   25756   
mean                    NaN          NaN     NaN        NaN    NaN     NaN   
std                     NaN          NaN     NaN        NaN    NaN     NaN   
min                     NaN          NaN     NaN        NaN    NaN     NaN   
25%                     NaN          NaN     NaN        NaN    NaN     NaN   
50%                     NaN          NaN     NaN        NaN    NaN     NaN   
75%                     NaN          NaN     NaN        NaN    NaN     NaN   
max                     NaN          NaN     NaN        NaN    NaN     NaN   

       vehicle_type  registration_year  gearbox   power_in_PS  

For columns of seller and offer_type, we can see that among several columns with only 2 unique values, these two have the frequncy of 49999, which means that almost all the values for these two columns are the same, so we can selectively exclude these two columns from our analysis later. Meanwhile, we also notice that all the data in the pictures_number column is 0, so we can also exclude this column.<br/>
Several other columns like last_seen, ad_created, and date_crawel may need further investigation due to the ambiguities in the summary, especially considering their different time formats.<br/>
The columns of price and odmeter are now store as the strings as the orginal data includes 'km' and '$', for which we should re-organize the data into numerical values and have the units listed in the column heads.

In [7]:
# Now we move on to transfer the data in columns of odmeter and price, as well as change the head name
# Remove all 'km' and '$' in these two columns
autos["odometer"] = autos["odometer"].str.replace("km", "").str.replace(",", "") \
.astype('int')
autos["price"] = autos["price"].str.replace("$", "").str.replace(",","") \
.astype('int')
# Change the head name
autos.rename({"odometer":"odometer_km"}, axis = 1, inplace = True)

In [8]:
# Check the data in the above two columns and see if there is any unreasonable data required to be removed
print(autos["price"].unique().shape, "\n")

print(autos["price"].describe(), "\n")

print(autos["price"].value_counts().sort_index(ascending = True).head(30), "\n")

print(autos["price"].value_counts().sort_index(ascending = True).tail(50), "\n")

print(autos["odometer_km"].unique().shape, "\n")

print(autos["odometer_km"].describe(), "\n")

print(autos["odometer_km"].value_counts().sort_index(ascending = True))

(2357,) 

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64 

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
40       6
45       4
47       1
49       4
50      49
55       2
59       1
60       9
65       5
66       1
Name: price, dtype: int64 

98500       1
99000       2
99900       2
104900      1
105000      2
109999      1
114400      1
115000      1
115991      1
116000      1
119500      1
119900      1
120000      2
128000      1
129000      1
130000      1
135000      1
137999      1
139997      1
145000      1
151990      1
155000      1
163500      1
163991      1
169000      1
169999      1
175000      1
180000      1
190000      1
194000      1
197000

From the observations above, firstly we can say that the data for odometer_km column look fine and we do not need to drop wring data. Then when it comes to the price column, we clearly see noticed that, there are in total 1421 vehicls sold at the price of 0, and many other vehicles sold at the price even lower than 100 as well as higher than 10,000,000, these data are questionable as usually, we seldom see used cars sold at such low/high prices and we as a result decide to removing these outliers.

In [9]:
# Removing the outliers
autos = autos[autos["price"].between(100, 300000)].copy()
#autos = autos.drop(autos[(autos["price"] < 100) | (autos["price"] > 300000)].index)
autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48222 entries, 0 to 49999
Data columns (total 20 columns):
date_crawled          48222 non-null object
name                  48222 non-null object
seller                48222 non-null object
offer_type            48222 non-null object
price                 48222 non-null int64
ab_test               48222 non-null object
vehicle_type          43799 non-null object
registration_year     48222 non-null int64
gearbox               46017 non-null object
power_in_PS           48222 non-null int64
model                 45828 non-null object
odometer_km           48222 non-null int64
registration_month    48222 non-null int64
fuel_type             44343 non-null object
brand                 48222 non-null object
unrepaired_damage     39338 non-null object
ad_created            48222 non-null object
pictures_number       48222 non-null int64
postal_code           48222 non-null int64
last_seen             48222 non-null object
dtypes: int64(7), 

In [10]:
print(autos["price"].describe(), "\n")
print(autos["price"].value_counts().sort_index(ascending = True).head(30), "\n")
print(autos["price"].value_counts().sort_index(ascending = True).tail(50), "\n")
print(autos["odometer_km"].describe(), "\n")
print(autos["odometer_km"].value_counts().sort_index(ascending = True))

count     48222.000000
mean       5916.204886
std        8807.999072
min         100.000000
25%        1250.000000
50%        3000.000000
75%        7499.000000
max      299000.000000
Name: price, dtype: float64 

100    134
110      3
111      2
115      2
117      1
120     39
122      1
125      8
129      1
130     15
135      1
139      1
140      9
145      2
149      7
150    224
156      2
160      8
170      7
173      1
175     12
179      1
180     35
185      1
188      1
190     16
193      1
195      2
198      1
199     41
Name: price, dtype: int64 

82987     1
83000     1
84000     1
84997     1
85000     1
86500     1
88900     1
89000     1
89900     1
93000     2
93911     1
94999     1
98500     1
99000     2
99900     2
104900    1
105000    2
109999    1
114400    1
115000    1
115991    1
116000    1
119500    1
119900    1
120000    2
128000    1
129000    1
130000    1
135000    1
137999    1
139997    1
145000    1
151990    1
155000    1
163500    1
163991  

After removing the outliers, we can see that for most of the used cars sold, the odometer is greater than 150,000 km, which accounts for more than 60%. The runner-ups are between 100,000 and 125,000 km, which accounts for more than 10%. For the odometer intervals with increment of 10,000 km from 20,000 to 90,000 km, they all lie in the interval between 700 and 1,800, as well as for cars with odometer less than 5,000 km. The average odometer is 125923 km, but this is only an approximation due to the estimated odometer for each car.<br\>
On the other hand, although the prices for the used car sold vary from 100 to 10,000,000, most of these lie in the interval between 1,000 and 30,000 (more than 95%). In our collected data, the average price for the used car is 5,916 and we can see that more than 75 of the car are cheaper than 7,500, but also higher than 1,250.

In [11]:
# Now we move on to deal with the columns having dates/times
# Columns of registration_month and registration year are already intergers
# Check the format of data_crawled, last_seen, and ad_created columns
autos[["date_crawled", "ad_created", "last_seen"]][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [12]:
# We will extract the year-month-date data and transform into datetime

autos["date_crawled"] = autos["date_crawled"].str.slice(0, 10)

# Transfrom this column into datetime format
autos["date_crawled"] = pd.to_datetime(autos["date_crawled"])

# Transform the ad_created and last_seen similarly
autos["ad_created"] = autos["ad_created"].str.slice(0, 10)
autos["last_seen"] = autos["last_seen"].str.slice(0, 10)
autos["ad_created"] = pd.to_datetime(autos["ad_created"])
autos["last_seen"] = pd.to_datetime(autos["last_seen"])

autos[["date_crawled", "ad_created", "last_seen"]][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26,2016-03-26,2016-04-06
1,2016-04-04,2016-04-04,2016-04-06
2,2016-03-26,2016-03-26,2016-04-06
3,2016-03-12,2016-03-12,2016-03-15
4,2016-04-01,2016-04-01,2016-04-01


In [ ]:
# Now we will be analyze these columns with their datetime formate
autos["date_crawled"].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025362
2016-03-06    0.014039
2016-03-07    0.036062
2016-03-08    0.033180
2016-03-09    0.033014
2016-03-10    0.032288
2016-03-11    0.032599
2016-03-12    0.036913
2016-03-13    0.015677
2016-03-14    0.036664
2016-03-15    0.034320
2016-03-16    0.029468
2016-03-17    0.031500
2016-03-18    0.012899
2016-03-19    0.034735
2016-03-20    0.037804
2016-03-21    0.037203
2016-03-22    0.032890
2016-03-23    0.032288
2016-03-24    0.029447
2016-03-25    0.031500
2016-03-26    0.032309
2016-03-27    0.031106
2016-03-28    0.034963
2016-03-29    0.034113
2016-03-30    0.033719
2016-03-31    0.031853
2016-04-01    0.033698
2016-04-02    0.035606
2016-04-03    0.038613
2016-04-04    0.036539
2016-04-05    0.013065
2016-04-06    0.003173
2016-04-07    0.001389
Name: date_crawled, dtype: float64

In [ ]:
autos["ad_created"].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033097
2016-03-10    0.031998
2016-03-11    0.032910
2016-03-12    0.036747
2016-03-13    0.017046
2016-03-14    0.035295
2016-03-15    0.034051
2016-03-16    0.029966
2016-03-17    0.031168
2016-03-18    0.013583
2016-03-19    0.033615
2016-03-20    0.037867
2016-03-21 

In [ ]:
autos["last_seen"].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001078
2016-03-06    0.004313
2016-03-07    0.005433
2016-03-08    0.007320
2016-03-09    0.009581
2016-03-10    0.010638
2016-03-11    0.012401
2016-03-12    0.023786
2016-03-13    0.008876
2016-03-14    0.012629
2016-03-15    0.015864
2016-03-16    0.016445
2016-03-17    0.028099
2016-03-18    0.007320
2016-03-19    0.015760
2016-03-20    0.020654
2016-03-21    0.020551
2016-03-22    0.021360
2016-03-23    0.018581
2016-03-24    0.019763
2016-03-25    0.019099
2016-03-26    0.016673
2016-03-27    0.015532
2016-03-28    0.020841
2016-03-29    0.022293
2016-03-30    0.024698
2016-03-31    0.023827
2016-04-01    0.022853
2016-04-02    0.024885
2016-04-03    0.025134
2016-04-04    0.024532
2016-04-05    0.125067
2016-04-06    0.221973
2016-04-07    0.132139
Name: last_seen, dtype: float64

We can see that, these data were crawled almost evenly from 2016-03-05 to 2016-04-07, with most of them sharing a percentage around 3%, except for few days with less than a 1.5% share.<br\>
The date that the advertisements were created ranges from 2015-06-11 to 2016-04-07, and for those dates before 2016-03, most of them only had one or two advertisements per day.<br\>
More than 50% of these Ads were last seen in April and dates of last seen for most other Ads are distributed quited evenly.

In [ ]:
# Look into the year of registration
autos["registration_year"].describe()

count    48222.000000
mean      2004.730579
std         87.899189
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Since the registration_year is already in the int format, we directly look into the statistics of that column, we observe that, the max and min of this column is 9999 and 1000, respectively. This indicates the existence of outlier as the year of cars shouldn't be expected to lie in this range.<br\>
We should expect to see cars registered no later than 2016 (when the list was studied) and no earlier than 1900 (it is very rare to find a car registered before that).

In [ ]:
# Similarly we drop the outliers
autos = autos[autos["registration_year"].between(1900, 2016)].copy()
autos["registration_year"].value_counts(normalize=True, dropna=False).head(20)

2000    0.066969
2005    0.062805
1999    0.062114
2004    0.058231
2003    0.058101
2006    0.057562
2001    0.056721
2002    0.053441
1998    0.050485
2007    0.049040
2008    0.047681
2009    0.044876
1997    0.041532
2011    0.034908
2010    0.034239
1996    0.029234
2012    0.028198
2016    0.025912
1995    0.025739
2013    0.017282
Name: registration_year, dtype: float64

We notice that year 1998-2008 account for the top 11 years with most registered cars and they vary from around 4.8% to 6.7%.

In [ ]:
# Check the top 20 brands of these used cars
autos["brand"].value_counts(normalize = True).head(15)

volkswagen       0.211413
bmw              0.110183
opel             0.107249
mercedes_benz    0.096656
audi             0.086775
ford             0.069838
renault          0.047077
peugeot          0.029860
fiat             0.025609
seat             0.018252
skoda            0.016419
nissan           0.015340
mazda            0.015232
smart            0.014196
citroen          0.014045
Name: brand, dtype: float64

We learn that there are in total 6 brands from Ford to Volkswagen whose share is greater than 5%, so we will work on those.

In [ ]:
mean_price_sum = {}
# Save the brands name we want to study in a list (pandas.index) to loop over
brands = autos["brand"].value_counts(normalize = True)
brands_list = brands[brands > 0.05].index

for brand in brands_list:
    brand_df = autos[autos["brand"] == brand]
    mean_price = brand_df["price"].mean()
    mean_price_sum[brand] = mean_price

mean_price_sum

Among all the brands that account for more than 5% in total sale cars, the Opel has the lowest sale price on average at around \$4,000, while ford has a relatively similarly \$3,700 average sale price. Meanwhile, the three luxury brands Audi, BMW, and Mercedes-Benz have the highest sale price on average, ranging from around \$8,400 to \$9,400. Volkswagen, who has the highest percentage in the car on sale, has the average sale price of \$5,437 in the middle.

In [ ]:
# In order to study the relation between the sale price and mileage, we will create a new dateframe to store all these info
# As it is difficult to store all info into one dictionary
# We choose to firstly create a dictionary to store the mileage info, and then transform two dicts into a dataframe
mean_mileage_sum = {}
for brand in brands_list:
    brand_df = autos[autos["brand"] == brand]
    mean_mileage = brand_df["odometer_km"].mean()
    mean_mileage_sum[brand] = mean_mileage

# Transform dictionary into series
price_series = pd.Series(mean_price_sum)
mileage_series = pd.Series(mean_mileage_sum)

# Construct the dataframe from series
comparison_df = pd.DataFrame({'Mean_Price':price_series, "Mean_Mileage":mileage_series})
comparison_df

As we can see here, the average mileages for different brands do not vary much as the prices, so these are not correlated.

In [ ]:
# Transfer German words into English words
print(autos.columns)
autos.head()

In [ ]:
print(autos["offer_type"].unique(), "\n")
print(autos["vehicle_type"].unique(), "\n")
print(autos["fuel_type"].unique(), "\n")
print(autos["unrepaired_damage"].unique(), "\n")
print(autos["gearbox"].unique())

In [ ]:
# Using Series.map() method
word_translate = {"Angebot":"offer", "bus":"bus", "limousine":"limousine", "kleinwagen":"Supermini", "kombi":"van",
                  "nan":"nan", "coupe":"coupe", "suv":"suv", "cabrio":"convertible", "andere":"other", "lpg":"lpg",
                  "benzin":"petrol", "diesel":"diesel", "cng":"cng", "hybrid":"hybrid", "elektro":"electro", "nein":"no",
                  "ja":"yes", "manuell":"manual", "automatik":"automatic"}

for category in ["offer_type", "vehicle_type", "fuel_type", "unrepaired_damage", "gearbox"]:
    autos[category] = autos[category].map(word_translate)

print(autos["offer_type"].unique(), "\n")
print(autos["vehicle_type"].unique(), "\n")
print(autos["fuel_type"].unique(), "\n")
print(autos["unrepaired_damage"].unique(), "\n")
print(autos["gearbox"].unique())

In [ ]:
# Change the datetime format into integer format
autos["date_crawled"] = autos["date_crawled"].astype(str).str.replace("-", "").astype(int)
autos["ad_created"] = autos["ad_created"].astype(str).str.replace("-", "").astype(int)
autos["last_seen"] = autos["last_seen"].astype(str).str.replace("-", "").astype(int)
autos.info()

In [ ]:
# Find the most common brand/model combinations
#most_common_combine = {}
most_common_model = {}
most_common_count = {}
# Save the brands name we want to study in a list (pandas.index) to loop over
brands_list = autos["brand"].unique()

for brand in brands_list:
    brand_df = autos[autos["brand"] == brand]
    models = brand_df["model"].value_counts(dropna = False).index.tolist()
    counts = brand_df["model"].value_counts(dropna = False).tolist()
    most_common_model[brand] = str(models[0])
    most_common_count[brand] = int(counts[0])
    #combine = brand + "+" + str(models[0])
    #most_common_combine[combine] = counts[0]

# Transform dictionary into series
model_series = pd.Series(most_common_model)
count_series = pd.Series(most_common_count)

# Construct the dataframe from series and sort by the counts
comparison_df = pd.DataFrame({'Model':most_common_model, "Count":most_common_count})
comparison_df.sort_values(by = "Count", ascending = False).head()

In [ ]:
# Study the relationship between the odometer and the average price
odo_price = {}

odometer_list = autos["odometer_km"].unique()

for odo in odometer_list:
    mean_price = autos.loc[autos["odometer_km"] == odo, "price"].mean()
    odo_price[odo] = mean_price

odo_price

In [ ]:
# Study the average price patterns based on the damages with cars
condition_price = {}

condition = autos["unrepaired_damage"].unique()
print(condition)
condition_exact = ["yes", "no"]

for c in condition_exact:
    mean_price = autos.loc[autos["unrepaired_damage"] == c, "price"].mean()
    condition_price[c] = mean_price
    
condition_price